# Business understanding

## Project overview

## Business problem

## Project objectives
### Main Objective

- The primary objective is to develop an NLP-based machine learning model that can accurately classify the sentiment of tweets related to Apple and Google products as positive, negative, or neutral.
### Specific Objectives



## Justification

## Questions 

# Data understanding

## Data Collection
The dataset for this project comes from [CrowdFlower](https://data.world/crowdflower/brands-and-product-emotions) and contains over 9,000 tweets labeled with sentiment (positive, negative, or neutral). These labeled examples provide a training set to build and evaluate the sentiment classification model. The data is enriched with various features such as tweet text, tweet ID, user ID, and the sentiment label, which is crucial for training the model.

To further improve the model, external data such as newer tweets about Apple and Google products or additional labeled sentiment datasets might be useful for retraining or fine-tuning.

### Load libraries

In [1]:
import pandas as pd
import re
import json
import emoji
import string
from nltk.corpus import stopwords 
from wordcloud import WordCloud
from nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
import warnings 
warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mich\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Load the data

In [2]:
df = pd.read_csv("data/tweet_product_company.csv", encoding="latin-1")
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [3]:
df.tail()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product
9092,Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...,NaN,No emotion toward brand or product


In [4]:
df.sample(5)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
4949,"40% of google maps usage is mobile, 150M users...",NaN,No emotion toward brand or product
2877,We have release Android App for Cluster -&gt; ...,NaN,No emotion toward brand or product
6768,RT @mention The simplest gesture on the ipad s...,NaN,No emotion toward brand or product
4720,"I don't know what time it is! iPhone, blackber...",NaN,No emotion toward brand or product
5156,RT @mention @mention Who said you won't be abl...,NaN,No emotion toward brand or product


#  Data cleaning

## Correct formats

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


- **tweet_text**: The data type `object` is appropriate as it contains text data.
- **emotion_in_tweet_is_directed_at**: The `object` type is suitable since it likely contains categorical or string values indicating emotion direction.
- **is_there_an_emotion_directed_at_a_brand_or_product**: The `object` type is correct as it likely holds categorical values (e.g., "Yes" or "No").


All columns have the correct data type (`object`) for their respective content.

## Missing Values

In [6]:
df.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

- The dataset has a total of 9093 rows.
- The **tweet_text** is the primary feature and is almost complete.
- The other two features have missing values and might require further preprocessing before use in sentiment analysis or classification tasks.



Removing rows with the missing value in **tweet_text** 

In [7]:
df.dropna(subset=["tweet_text"],inplace=True)
df.isna().sum()

tweet_text                                               0
emotion_in_tweet_is_directed_at                       5801
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

Replacing missing values on column **emotion_in_tweet_is_directed_at** with `'Unknown'`

In [8]:
product_names = df.emotion_in_tweet_is_directed_at.unique()
df.emotion_in_tweet_is_directed_at.fillna("Unknown",inplace=True)
df.isna().sum() 

tweet_text                                            0
emotion_in_tweet_is_directed_at                       0
is_there_an_emotion_directed_at_a_brand_or_product    0
dtype: int64

Keeping the original text

In [9]:
df["original_tweet"]= df.tweet_text

## Feature Engineering

### Removing Garbled Text

In [10]:
# Function to clean garbled characters (non-ASCII or unwanted characters)
def remove_garbled_text(text):
    if isinstance(text, str): 
        return re.sub(r'[^\x00-\x7F]+', '', text)
    return text

# Apply this function to your text data
df['tweet_text'] = df['tweet_text'].apply(remove_garbled_text)

df.tweet_text

0       .@wesley83 I have a 3G iPhone. After 3 hrs twe...
1       @jessedee Know about @fludapp ? Awesome iPad/i...
2       @swonderlin Can not wait for #iPad 2 also. The...
3       @sxsw I hope this year's festival isn't as cra...
4       @sxtxstate great stuff on Fri #SXSW: Marissa M...
                              ...                        
9088                        Ipad everywhere. #SXSW {link}
9089    Wave, buzz... RT @mention We interrupt your re...
9090    Google's Zeiger, a physician never reported po...
9091    Some Verizon iPhone customers complained their...
9092    ___RT @mention Google Tests Check-in Offers At...
Name: tweet_text, Length: 9092, dtype: object

### Lowercasing
Convert all `tweet_text` to lowercase to maintain consistency

In [11]:
df.tweet_text = df.tweet_text.str.lower()

**Converting Transcript to strings**

In [12]:
def transcription_to_strings(df): 
    # matches basic sad face :(, :-(
    sad_face = re.compile(r'[:;8]?[\'\-]?\([/\\]')  
    
    # matches crying face :'(, :'-(
    crying_face = re.compile(r'[:;8]?[\'\-]?\'\(')  
    
    # matches complex sad faces with slashes, tears, etc.
    complex_sad_face = re.compile(r'[:;8]?[\'\-]?\([/\\]\)?')  
    
    # edge case: faces with tears or creative symbols like T_T, TT_TT
    crying_edge_case = re.compile(r'(T_T|tt_t|TT_TT)')
    
    # apply pattern replacement across the DataFrame's text column
    df['tweet_text'] = df['tweet_text'].apply(lambda x: 
        crying_edge_case.sub('crying face', 
        complex_sad_face.sub('complex sad face', 
        crying_face.sub('crying face', 
        sad_face.sub('sad face', x)))))

transcription_to_strings(df)
    

**Dealing with emojis**

In [13]:
for index in range(df.tweet_text.shape[0]):
    df.tweet_text.iloc[index] = emoji.demojize(df.tweet_text.iloc[index])

**Replacing abbreviation with its full form**

In [14]:
# loading abbreviation dictionary from abbr.txt
with open('abbr.txt', 'r') as file:
    abbr_dict = json.load(file)

# expand abbreviations in a tweet
def expand_abbr_in_tweet(text):
    return " ".join([abbr_dict.get(word, word) for word in text.split()])

# apply the function to each tweet in the DataFrame
df['tweet_text'] = df['tweet_text'].apply(expand_abbr_in_tweet)

### Removing Links
First check if there is any links

In [15]:
def text_processor(process, pattern):   
    def extract_pattern(text):
        if isinstance(text, str):
            return re.findall(pattern, text)
        return []

    # apply pattern extraction
    links_list = df['tweet_text'].apply(extract_pattern)
    
    # flatten the list of lists
    all_links = [link for sublist in links_list for link in sublist]

    # check if anything was found
    if all_links:
        print(f"{process} found in the dataset: {len(all_links)} matches")
        print(all_links[:5])  # optional: show a few samples
    else:
        print(f"No {process} found.")

In [16]:
def remove_pattern(pattern):
    df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub(pattern, '', x))

In [17]:
to_remove_or_check = "Links"
pattern = r'http\S+|www\S+'
text_processor(to_remove_or_check,pattern)

Links found in the dataset: 48 matches
['http://ht.ly/49n4m', 'http://bit.ly/ieavob', 'http://bit.ly/gvlrin', 'http://j.mp/grn7pk)', 'http://bit.ly/axzwxb']


Removing links found

In [18]:
remove_pattern(pattern)

Checking if any links are remaining

In [19]:
text_processor(to_remove_or_check,pattern)

No Links found.


**Removing Usernames**

Checking for twitter usernames

In [20]:
to_remove_or_check = "Usernames"
pattern = r'@\S+'
text_processor(to_remove_or_check,pattern)

Usernames found in the dataset: 7192 matches
['@wesley83', '@jessedee', '@fludapp', '@swonderlin', '@sxsw']


Removing Usernames

In [21]:
remove_pattern(pattern)

Checking if any usernames are remaining


In [22]:
text_processor(to_remove_or_check,pattern)

No Usernames found.


**Removing Hashtags**

Let us first check if there is any hashtags

In [23]:
to_remove_or_check = "Hashtags"
pattern = r'#\S+'
text_processor(to_remove_or_check,pattern)

Hashtags found in the dataset: 15853 matches
['#rise_austin,', '#sxsw.', '#sxsw', '#ipad', '#sxsw.']


Removing hashtags

In [24]:
remove_pattern(pattern)

In [25]:
text_processor(to_remove_or_check,pattern)

No Hashtags found.


**Removing Punctuation**

In [26]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,original_tweet
0,. i have a 3g iphone. after 3 hrs tweeting at ...,iPhone,Negative emotion,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,know about ? awesome ipad/iphone application...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad/i...
2,can not wait for 2 also. they should sale th...,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also. The...
3,i hope this year's festival isn't as crashy a...,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,"great stuff on fri marissa mayer (google), t...",Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW: Marissa M...


In [27]:
# function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# applying the function to the tweet_text column
df['tweet_text'] = df['tweet_text'].apply(remove_punctuation)

In [28]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,original_tweet
0,i have a 3g iphone after 3 hrs tweeting at i...,iPhone,Negative emotion,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,know about awesome ipadiphone application t...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad/i...
2,can not wait for 2 also they should sale the...,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also. The...
3,i hope this years festival isnt as crashy as ...,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,great stuff on fri marissa mayer google tim ...,Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW: Marissa M...


**Removing Stopwords**

In [29]:
product_names = list(product_names) 

In [30]:
#adding product names when there combined to accomodate such instances 
product_names_joined = [str(name1).lower() + str(name2).lower() for name1 in product_names 
                        for name2 in product_names if not pd.isna(name1) or not pd.isna(name2)]
product_names.extend(product_names_joined)
# converting all product names to lowercase
product_names= [str(name).lower() for name in product_names]


In [31]:
# getting stopwords 
stopwords_list = stopwords.words('english')

#removing punctuation from stopwords since punctuation have been removed
stopwords_list= [word.translate(str.maketrans('', '', string.punctuation)) for word in stopwords_list]

# extending the list with lowercase product names
stopwords_list.extend(product_names)

In [32]:
for index in range(df.tweet_text.shape[0]):
    tweet_text = df.tweet_text.iloc[index].split()
    #removing stopwords from text
    tweet_no_stopword = " ".join([word for word in tweet_text if word not in stopwords_list])
    df.tweet_text.iloc[index] = tweet_no_stopword

In [33]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,original_tweet
0,3g 3 hrs tweeting dead need upgrade plugin sta...,iPhone,Negative emotion,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,know awesome application likely appreciate des...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad/i...
2,wait 2 also sale,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also. The...
3,hope years festival crashy years app,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,great stuff fri marissa mayer tim oreilly tech...,Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW: Marissa M...


**Removing Numbers**

In [34]:
for index in range(df.tweet_text.shape[0]):
    tweet = df.tweet_text.iloc[index].split()
    df.tweet_text.iloc[index] = " ".join([re.sub(r'\b\d+\b', '', text) for text in tweet])

In [35]:
df[["tweet_text","original_tweet"]].head(10)

,tweet_text,original_tweet
0,3g hrs tweeting dead need upgrade plugin stat...,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,know awesome application likely appreciate des...,@jessedee Know about @fludapp ? Awesome iPad/i...
2,wait also sale,@swonderlin Can not wait for #iPad 2 also. The...
3,hope years festival crashy years app,@sxsw I hope this year's festival isn't as cra...
4,great stuff fri marissa mayer tim oreilly tech...,@sxtxstate great stuff on Fri #SXSW: Marissa M...
5,new applications communication showcased confe...,@teachntech00 New iPad Apps For #SpeechTherapy...
7,starting around corner hop skip jump good time...,"#SXSW is just starting, #CTIA is around the co..."
8,beautifully smart simple idea retweet wrote ap...,Beautifully smart and simple idea RT @madebyma...
9,counting days plus strong canadian dollar mean...,Counting down the days to #sxsw plus strong Ca...
10,excited meet show sprint galaxy still running,Excited to meet the @samsungmobileus at #sxsw ...


### Tokenization

In [36]:
for index in range(df.tweet_text.shape[0]):
    df.tweet_text.iloc[index] = word_tokenize(df.tweet_text.iloc[index])

In [37]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,original_tweet
0,"[3g, hrs, tweeting, dead, need, upgrade, plugi...",iPhone,Negative emotion,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,"[know, awesome, application, likely, appreciat...",iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad/i...
2,"[wait, also, sale]",iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also. The...
3,"[hope, years, festival, crashy, years, app]",iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,"[great, stuff, fri, marissa, mayer, tim, oreil...",Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW: Marissa M...


### Lemmatization

In [38]:
lemmatizer = WordNetLemmatizer()

for index in range(df.tweet_text.shape[0]):
    text = df.tweet_text.iloc[index] 
    tweet_text =[lemmatizer.lemmatize(word) for word in text]
    df.tweet_text.iloc[index] = tweet_text

In [39]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,original_tweet
0,"[3g, hr, tweeting, dead, need, upgrade, plugin...",iPhone,Negative emotion,.@wesley83 I have a 3G iPhone. After 3 hrs twe...
1,"[know, awesome, application, likely, appreciat...",iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad/i...
2,"[wait, also, sale]",iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also. The...
3,"[hope, year, festival, crashy, year, app]",iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,"[great, stuff, fri, marissa, mayer, tim, oreil...",Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW: Marissa M...


In [41]:
# Prepare text (assuming df.tweet_text is a list of lists of tokens)
texts = [" ".join(text) for text in df.tweet_text]

# CountVectorizer
count_vect = CountVectorizer()
count_matrix = count_vect.fit_transform(texts)
count_df = pd.DataFrame(count_matrix.toarray(), columns=count_vect.get_feature_names())
print("CountVectorizer:\n")
print(count_df)

# TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_matrix = tfidf_vect.fit_transform(texts)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vect.get_feature_names())

<IPython.core.display.Javascript object>

CountVectorizer:

      0310apple  100  103011p  1045am3  10am  10k  10mins  10pm  10x  10x2  \
0             0    0        0        0     0    0       0     0    0     0   
1             0    0        0        0     0    0       0     0    0     0   
2             0    0        0        0     0    0       0     0    0     0   
3             0    0        0        0     0    0       0     0    0     0   
4             0    0        0        0     0    0       0     0    0     0   
...         ...  ...      ...      ...   ...  ...     ...   ...  ...   ...   
9087          0    0        0        0     0    0       0     0    0     0   
9088          0    0        0        0     0    0       0     0    0     0   
9089          0    0        0        0     0    0       0     0    0     0   
9090          0    0        0        0     0    0       0     0    0     0   
9091          0    0        0        0     0    0       0     0    0     0   

      ...  zite  zlf  zms  zombie  zomg  zone

<IPython.core.display.Javascript object>

In [42]:
top_n = 5  # change this to however many you want

print("\nTF-IDF Vectorizer (Top Words per Document):\n")
for i, row in tfidf_df.iterrows():
    doc_tfidf = row[row > 0].sort_values(ascending=False).head(top_n).round(3)
    print(f"\nDocument {i + 1}:")
    print(doc_tfidf.reset_index().rename(columns={'index': 'Word', i: 'TF-IDF'}))


TF-IDF Vectorizer (Top Words per Document):


Document 1:
      Word  TF-IDF
0       hr   0.413
1   plugin   0.413
2  station   0.367
3  upgrade   0.363
4     dead   0.351

Document 2:
         Word  TF-IDF
0  appreciate   0.479
1      likely   0.430
2      giving   0.336
3        also   0.332
4      design   0.304

Document 3:
   Word  TF-IDF
0  sale   0.610
1  wait   0.561
2  also   0.560

Document 4:
       Word  TF-IDF
0      year   0.594
1    crashy   0.527
2  festival   0.387
3      hope   0.356
4       app   0.305

Document 5:
               Word  TF-IDF
0  booksconferences   0.361
1         mullenweg   0.361
2               fri   0.322
3         wordpress   0.322
4              matt   0.309

Document 6:
            Word  TF-IDF
0      showcased   0.552
1  communication   0.545
2     conference   0.515
3    application   0.269
4            new   0.246

Document 7:
     Word  TF-IDF
0     hop   0.420
1    skip   0.420
2    jump   0.393
3  corner   0.353
4     fan   0.318

Docume

       Word  TF-IDF
0  headline   0.407
1       hmm   0.389
2  musthave   0.389
3    gadget   0.360
4      quot   0.316

Document 102:
        Word  TF-IDF
0     opened   0.423
1       town   0.416
2       nerd   0.396
3  quotapple   0.375
4      ipads   0.338

Document 103:
              Word  TF-IDF
0  quotdatavizquot   0.504
1  quotsatanicquot   0.504
2       translates   0.504
3            sayin   0.413
4             know   0.261

Document 104:
       Word  TF-IDF
0     check   0.400
1   agoquot   0.367
2       out   0.367
3        in   0.358
4  checkins   0.296

Document 105:
       Word  TF-IDF
0     check   0.395
1   agoquot   0.362
2       out   0.362
3        in   0.353
4  checkins   0.291

Document 106:
        Word  TF-IDF
0  quotthink   0.399
1  tweetquot   0.399
2       mark   0.394
3   belinsky   0.387
4  speakquot   0.387

Document 107:
       Word  TF-IDF
0    search   0.561
1     brain   0.497
2  quotleft   0.311
3     xquot   0.311
4    engine   0.249

Document 108:
 

4     major   0.283

Document 216:
           Word  TF-IDF
0  kiiiiiilling   0.428
1       pissing   0.428
2          piss   0.414
3          took   0.329
4         trade   0.315

Document 217:
      Word  TF-IDF
0   monger   0.602
1     read   0.421
2  preview   0.408
3    rumor   0.335
4     good   0.319

Document 218:
      Word  TF-IDF
0  shotgun   0.647
1     hobo   0.627
2     game   0.410
3     link   0.141

Document 219:
      Word  TF-IDF
0  shotgun   0.461
1     hobo   0.447
2   please   0.336
3      yes   0.330
4   coming   0.309

Document 220:
      Word  TF-IDF
0    build   0.422
1  history   0.405
2  biggest   0.385
3       le   0.373
4  company   0.317

Document 221:
            Word  TF-IDF
0        instant   0.534
1      starbucks   0.534
2  gratification   0.280
3         survey   0.273
4       research   0.254

Document 222:
        Word  TF-IDF
0        5pm   0.489
1     retail   0.485
2    release   0.419
3       open   0.314
4  temporary   0.310

Document 223:
   


Document 278:
      Word  TF-IDF
0  opening   0.534
1     sxsw   0.460
2    popup   0.438
3   austin   0.397
4    store   0.331

Document 279:
       Word  TF-IDF
0  geekfest   0.679
1   opening   0.392
2      sxsw   0.338
3     popup   0.322
4    austin   0.292

Document 280:
       Word  TF-IDF
0   crossed   0.383
1     fresh   0.373
2      noon   0.358
3  shipment   0.352
4    finger   0.342

Document 281:
    Word  TF-IDF
0   save   0.585
1    set   0.438
2   open   0.400
3   sxsw   0.341
4  popup   0.325

Document 282:
        Word  TF-IDF
0     expert   0.591
1     school   0.569
2  marketing   0.549
3       link   0.159

Document 283:
        Word  TF-IDF
0     expert   0.591
1     school   0.569
2  marketing   0.549
3       link   0.159

Document 284:
        Word  TF-IDF
0     expert   0.556
1     school   0.536
2  marketing   0.516
3        via   0.340
4       link   0.150

Document 285:
        Word  TF-IDF
0  marketing   0.538
1     loathe   0.413
2       ltlt   0.365
3   


Document 340:
    Word  TF-IDF
0  queue   0.799
1  popup   0.448
2  store   0.339
3   link   0.213

Document 341:
    Word  TF-IDF
0   four   0.464
1  crazy   0.403
2  queue   0.397
3  stuff   0.387
4  block   0.355

Document 342:
         Word  TF-IDF
0         wif   0.352
1         dmi   0.337
2    intimate   0.318
3      posted   0.318
4  restaurant   0.305

Document 343:
       Word  TF-IDF
0   chelsea   0.442
1   handler   0.442
2   sounded   0.423
3  laughing   0.334
4     woman   0.330

Document 344:
       Word  TF-IDF
0   hauling   0.394
1   laughed   0.394
2       rub   0.394
3  shoulder   0.365
4     spent   0.302

Document 345:
       Word  TF-IDF
0  optiscan   0.400
1     image   0.346
2        qr   0.343
3    reader   0.334
4     badge   0.322

Document 346:
           Word  TF-IDF
0  attendeesyou   0.459
1           hop   0.426
2          four   0.381
3       learned   0.336
4           hey   0.269

Document 347:
        Word  TF-IDF
0        plz   0.476
1       aclu   

...

      Word  TF-IDF
0  lecture   0.551
1    stove   0.551
2     turn   0.393
3    phone   0.314
4    first   0.293

Document 8165:
            Word  TF-IDF
0         lifter   0.557
1        h4ckers   0.503
2  extraordinary   0.413
3         league   0.413
4         google   0.290

Document 8166:
          Word  TF-IDF
0    sponsored   0.519
1     includes   0.492
2    uberguide   0.492
3  application   0.289
4          new   0.264

Document 8167:
        Word  TF-IDF
0  partygood   0.579
1    hanging   0.509
2   industry   0.443
3      crowd   0.431
4    retweet   0.153

Document 8168:
         Word  TF-IDF
0      agreed   0.394
1      afford   0.337
2    consider   0.330
3      attend   0.321
4  earthquake   0.319

Document 8169:
     Word  TF-IDF
0   aisle   0.528
1  across   0.417
2   visit   0.402
3   trade   0.390
4    show   0.306

Document 8170:
       Word  TF-IDF
0  exchange   0.423
1   branded   0.414
2      woah   0.406
3    tattoo   0.362
4     crazy   0.337

Document 8171:


4    effective   0.348

Document 8224:
      Word  TF-IDF
0  exhibit   0.593
1    daily   0.548
2   detail   0.495
3   austin   0.285
4     link   0.149

Document 8225:
            Word  TF-IDF
0     actionable   0.360
1  analyticsquot   0.360
2        cricket   0.360
3         metric   0.334
4       quotwhat   0.325

Document 8226:
       Word  TF-IDF
0  anything   0.687
1       buy   0.551
2     popup   0.378
3     store   0.285

Document 8227:
                 Word  TF-IDF
0  superhappydevhouse   0.476
1             hacking   0.440
2                rsvp   0.440
3              sunday   0.399
4              enough   0.376

Document 8228:
     Word  TF-IDF
0     cuz   0.401
1   clear   0.372
2  saying   0.351
3   funny   0.332
4    stay   0.332

Document 8229:
       Word  TF-IDF
0    buzzrt   0.570
1      feel   0.389
2  possibly   0.279
3     major   0.267
4    called   0.256

Document 8230:
        Word  TF-IDF
0      music   0.597
1       free   0.477
2      track   0.347
3    samp


Document 8323:
         Word  TF-IDF
0    recycled   0.527
1  skateboard   0.527
2        oooh   0.504
3        case   0.299
4        want   0.277

Document 8324:
    Word  TF-IDF
0   aapl   0.565
1   sale   0.442
2  smart   0.418
3    pop   0.355
4   open   0.324

Document 8325:
      Word  TF-IDF
0     damm   0.611
1    smart   0.386
2     temp   0.374
3  opening   0.296
4  retweet   0.291

Document 8326:
       Word  TF-IDF
0  chilling   0.434
1      hair   0.387
2   station   0.363
3    maggie   0.332
4     night   0.284

Document 8327:
     Word  TF-IDF
0  faulty   0.662
1   whats   0.460
2     app   0.384
3      go   0.344
4    sxsw   0.289

Document 8328:
      Word  TF-IDF
0       rt   0.499
1    major   0.368
2   called   0.353
3  network   0.332
4   circle   0.321

Document 8329:
              Word  TF-IDF
0             true   0.350
1  conferencesquot   0.349
2        quotapple   0.327
3       technology   0.324
4               rt   0.320

Document 8330:
        Word  TF-IDF

          Word  TF-IDF
0     download   0.811
1  application   0.520
2         link   0.269

Document 8418:
        Word  TF-IDF
0  bythepool   0.301
1     cowboy   0.301
2  hollywood   0.301
3    silicon   0.301
4       vibe   0.301

Document 8419:
     Word  TF-IDF
0   jamin   0.698
1   album   0.544
2  itunes   0.446
3    link   0.132

Document 8420:
           Word  TF-IDF
0  r3conf1gur3d   0.388
1     soundtrax   0.388
2    tronlegacy   0.388
3        disney   0.366
4      checkout   0.336

Document 8421:
        Word  TF-IDF
0        awe   0.393
1    jetsons   0.393
2  inspiring   0.364
3  rendering   0.364
4         3d   0.302

Document 8422:
     Word  TF-IDF
0   crack   0.558
1  addict   0.534
2   house   0.414
3     mac   0.402
4    like   0.264

Document 8423:
      Word  TF-IDF
0  waiting   0.420
1      buy   0.373
2    ipad2   0.344
3     want   0.339
4      see   0.321

Document 8424:
         Word  TF-IDF
0  protecting   0.451
1       guard   0.413
2    security   0.373


KeyboardInterrupt: 

# Preprocessing

# Modeling

# Evaluation

# Conclusion